In [32]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.basics import Instrument
from qubx.impl.ccxt_connector import CCXT_connector

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
lookup.instruments['KRAKEN.F:BTC/USD']

[Instrument(symbol='BTC/USD:BTC', market_type='swap', exchange='KRAKEN.F', base='BTC', quote='USD', margin_symbol='BTC', min_tick=0.5, min_size_step=0.5, min_size=1.0, futures_info=FuturesInfo(contract_type='futures_inverse', delivery_date=datetime.datetime(2100, 1, 1, 0, 0), onboard_date=datetime.datetime(2018, 8, 31, 0, 0), contract_size='1', maint_margin=0.01, required_margin=0.02, liquidation_fee=0.0), _aux_instrument=None),
 Instrument(symbol='BTC/USD:USD', market_type='swap', exchange='KRAKEN.F', base='BTC', quote='USD', margin_symbol='USD', min_tick=1.0, min_size_step=1.0, min_size=0.0001, futures_info=FuturesInfo(contract_type='flexible_futures', delivery_date=datetime.datetime(2100, 1, 1, 0, 0), onboard_date=datetime.datetime(2022, 3, 22, 13, 15, 36), contract_size='1', maint_margin=0.01, required_margin=0.02, liquidation_fee=0.0), _aux_instrument=None),
 Instrument(symbol='BTC/USD:BTC-240329', market_type='future', exchange='KRAKEN.F', base='BTC', quote='USD', margin_symbol='

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [49]:
interval = bu.parse_timeframe('1m') * 1000
limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

[[1710510720000, 3743.41, 3744.09, 3732.57, 3735.19, 5135.436],
 [1710510780000, 3735.2, 3735.91, 3731.0, 3732.98, 2215.002],
 [1710510840000, 3732.99, 3733.66, 3728.59, 3731.65, 2183.149],
 [1710510900000, 3731.65, 3740.76, 3729.49, 3735.22, 3754.617],
 [1710510960000, 3735.23, 3737.89, 3729.04, 3729.53, 2323.117],
 [1710511020000, 3729.54, 3729.74, 3722.39, 3725.59, 3916.295],
 [1710511080000, 3725.59, 3726.92, 3718.36, 3722.38, 4205.346],
 [1710511140000, 3722.38, 3727.25, 3722.38, 3726.53, 2059.095],
 [1710511200000, 3726.54, 3735.0, 3725.4, 3729.97, 4562.525],
 [1710511260000, 3729.98, 3729.98, 3721.07, 3721.29, 2894.243],
 [1710511320000, 3721.3, 3721.62, 3707.78, 3713.09, 6533.614],
 [1710511380000, 3713.1, 3721.92, 3711.69, 3721.42, 2548.088],
 [1710511440000, 3721.42, 3721.88, 3711.86, 3715.75, 2467.861],
 [1710511500000, 3715.75, 3715.96, 3705.09, 3708.09, 3732.024],
 [1710511560000, 3708.08, 3709.03, 3704.8, 3707.45, 2758.891],
 [1710511620000, 3707.46, 3709.73, 3696.12, 369

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [ ]:
bnc.market(instr.symbol)
krk.market('PF_ETHUSD')

In [24]:
# bnc.market_id('BTC/USDT')

'BTCUSDT'

In [66]:
await bnc.

{'BTC/USDT:USDT': {'symbol': 'BTC/USDT:USDT',
  'timestamp': None,
  'datetime': None,
  'high': None,
  'low': None,
  'bid': 67795.4,
  'bidVolume': 0.007,
  'ask': 67795.5,
  'askVolume': 4.016,
  'vwap': None,
  'open': None,
  'close': None,
  'last': None,
  'previousClose': None,
  'change': None,
  'percentage': None,
  'average': None,
  'baseVolume': None,
  'quoteVolume': None,
  'info': {'symbol': 'BTCUSDT',
   'bidPrice': '67795.40',
   'bidQty': '0.007',
   'askPrice': '67795.50',
   'askQty': '4.016',
   'time': '1710509128176',
   'lastUpdateId': '4189763397398'}}}

In [48]:
# r = await bnc.public_get_klines(params)
r1 = await krk.fetch_ohlcv('PF_ETHUSD', '1m', since=since, limit=limit)

In [49]:
r1

[[1710501960000, 3640.0, 3641.8, 3636.3, 3636.3, 2.212],
 [1710502020000, 3636.3, 3641.6, 3636.1, 3638.3, 6.139],
 [1710502080000, 3638.3, 3642.2, 3635.3, 3635.9, 10.526],
 [1710502140000, 3635.9, 3638.1, 3630.0, 3630.0, 0.734],
 [1710502200000, 3630.0, 3640.9, 3630.0, 3640.9, 3.586999999999999],
 [1710502260000, 3640.9, 3651.8, 3640.9, 3649.1, 12.55],
 [1710502320000, 3649.1, 3656.1, 3647.3, 3655.2, 22.014],
 [1710502380000, 3655.2, 3655.2, 3652.5, 3652.5, 3.99],
 [1710502440000, 3652.5, 3660.0, 3650.6, 3660.0, 3.249],
 [1710502500000, 3660.0, 3668.0, 3659.0, 3660.1, 6.283999999999998],
 [1710502560000, 3660.1, 3662.4, 3654.3, 3662.4, 107.676],
 [1710502620000, 3662.4, 3668.4, 3661.4, 3667.6, 7.918],
 [1710502680000, 3667.6, 3675.9, 3665.7, 3673.4, 43.353],
 [1710502740000, 3673.4, 3674.9, 3668.4, 3672.7, 4.592000000000001],
 [1710502800000, 3672.7, 3676.4, 3670.9, 3675.2, 5.71],
 [1710502860000, 3675.2, 3675.2, 3667.3, 3669.8, 1.422],
 [1710502920000, 3669.8, 3674.0, 3668.2, 3673.7, 

In [35]:
pd.Timestamp(bnc.microseconds(), unit='us')

Timestamp('2024-03-12 14:30:12.438571')

In [57]:
cxc = CCXT_connector('binance')

In [ ]:

params = { 
    'symbol': instr.symbol, 
    'interval': bnc.timeframes['1m'],
}